# LangChain Expression Language (LCEL) Tutorial

In [1]:
# For YouTube Educational Content
from typing import Any, Dict, List

from dotenv import load_dotenv
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser

# Import LangChain components
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_ollama import ChatOllama


/Users/james/miniconda3/envs/py312/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


## Setting things up

In [ ]:
# Load environment variables
load_dotenv()

# Initialize the LLM
llm = ChatOllama(model="llama3.2",temperature=0.3)


## SECTION 1: Basic LCEL Chain

In [3]:

print("-" * 50,'\n',"DEMO 1: Basic LCEL Chain",'\n',"-" * 50)

# Create a simple prompt template
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}.")

# Create a basic chain using the | operator (pipe)
basic_chain = prompt | llm | StrOutputParser()

# Run the chain
result = basic_chain.invoke({"topic": "programming"})
print(result)
print("\n")



-------------------------------------------------- 
 DEMO 1: Basic LCEL Chain 
 --------------------------------------------------
Why do programmers prefer dark mode?

Because light attracts bugs.




In [4]:

# ###############################################################
# SECTION 2: Parallel Processing with LCEL
# ###############################################################

print("-" * 50,'\n',"DEMO 2: Parallel Processing with LCEL",'\n',"-" * 50)

# Create two different prompts
pros_prompt = ChatPromptTemplate.from_template(
    "List 3 pros of {technology} technology."
)
cons_prompt = ChatPromptTemplate.from_template(
    "List 3 cons of {technology} technology."
)

# Create chains for each prompt
pros_chain = pros_prompt | llm | StrOutputParser()
cons_chain = cons_prompt | llm | StrOutputParser()

# Create a parallel chain
parallel_chain = RunnableParallel(pros=pros_chain, cons=cons_chain)

# Run the parallel chain
parallel_result = parallel_chain.invoke({"technology": "blockchain"})
print("PROS:")
print(parallel_result["pros"])
print("\nCONS:")
print(parallel_result["cons"])
print("\n")



-------------------------------------------------- 
 DEMO 2: Parallel Processing with LCEL 
 --------------------------------------------------
PROS:
Here are three pros of blockchain technology:

1. **Security and Transparency**: Blockchain technology uses a decentralized, distributed ledger that records transactions across multiple computers. This makes it virtually impossible to alter or manipulate the data once it's been recorded, ensuring the integrity and security of the information.

2. **Immutable and Tamper-Proof**: The use of cryptography and complex algorithms in blockchain technology ensures that all transactions are time-stamped and linked together, creating a permanent record that cannot be altered or deleted. This makes it an ideal solution for applications where data needs to be tamper-proof.

3. **Decentralized and Autonomous**: Blockchain technology operates independently of central authorities, allowing for peer-to-peer transactions without the need for intermediarie

In [5]:
# %%
# ###############################################################
# SECTION 3: Advanced Chain with Data Transformation
# ###############################################################
print("-" * 50,'\n',"DEMO 3: Advanced Chain with Data Transformation",'\n',"-" * 50)

# Define a function to process input
def preprocess_input(input_data: Dict[str, Any]) -> Dict[str, Any]:
    """Process the input data before sending to the LLM."""
    # Add a timestamp, capitalize the query, etc.
    processed_data = input_data.copy()
    processed_data["query"] = input_data["query"].upper()
    processed_data["word_count"] = len(input_data["query"].split())
    return processed_data


# Define a function to process output
def postprocess_output(output: str) -> Dict[str, Any]:
    """Process the output from the LLM."""
    lines = output.strip().split("\n")
    return {
        "summary": lines[0] if lines else "",
        "details": lines[1:],
        "response_length": len(output),
    }


# Create an advanced prompt
advanced_prompt = ChatPromptTemplate.from_template(
    """Answer the following query: {query}

    The query has {word_count} words.
    Provide a detailed explanation.
    """
)

# Create an advanced chain with preprocessing and postprocessing
advanced_chain = (
    RunnableLambda(preprocess_input)
    | advanced_prompt
    | llm
    | StrOutputParser()
    | RunnableLambda(postprocess_output)
)

# Run the advanced chain
advanced_result = advanced_chain.invoke(
    {"query": "how does LCEL improve LLM applications"}
)
print(f"Summary: {advanced_result['summary']}")
print("Details:")
for detail in advanced_result["details"]:
    if detail.strip():
        print(f"- {detail.strip()}")
print(f"Response Length: {advanced_result['response_length']} characters")
print("\n")



-------------------------------------------------- 
 DEMO 3: Advanced Chain with Data Transformation 
 --------------------------------------------------
Summary: LCE (Local Consistency Estimation) is a technique used to improve Large Language Model (LLM) applications. Here's a detailed explanation of how LCE improves LLM applications:
Details:
- **What is Local Consistency Estimation (LCE)?**
- LCE is a method for estimating the consistency of local contexts in a sequence of tokens. In natural language processing, local context refers to the surrounding words or tokens that influence the meaning of a particular word. LCE estimates the probability distribution over these local contexts, allowing models to better understand the relationships between tokens and make more accurate predictions.
- **How does LCE improve LLM applications?**
- LCE improves LLM applications in several ways:
- 1. **Improved contextual understanding**: By estimating the consistency of local contexts, LCE helps m

In [6]:
# %%
# ###############################################################
# SECTION 4: RAG Pattern with LCEL
# ###############################################################
print("-" * 50,'\n',"DEMO 4: RAG Pattern with LCEL (Simulated)",'\n',"-" * 50)


# Simulate a retriever function
def simulate_retrieval(query: str) -> List[str]:
    """Simulate document retrieval based on query."""
    # In a real application, this would query a vector database
    documents = [
        "LCEL allows for declarative chain construction in LangChain.",
        "LangChain Expression Language simplifies building complex LLM applications.",
        "LCEL supports streaming, async operations, and parallel execution.",
    ]
    return documents


# Create a RAG prompt
rag_prompt = ChatPromptTemplate.from_template(
    """Answer the question based on the following context:

    Context:
    {context}

    Question: {question}
    """
)


# Create a RAG chain
def format_docs(docs):
    return "\n".join(docs)


rag_chain = (
    RunnableParallel(
        {
            "context": RunnableLambda(simulate_retrieval) | RunnableLambda(format_docs),
            "question": RunnablePassthrough(),
        }
    )
    | rag_prompt
    | llm
    | StrOutputParser()
)

# %%
# Run the RAG chain
rag_result = rag_chain.invoke("What is LCEL and why is it useful?")
print(rag_result)

# %%

-------------------------------------------------- 
 DEMO 4: RAG Pattern with LCEL (Simulated) 
 --------------------------------------------------
Based on the given context, LCEL stands for LangChain Expression Language. It's a tool that allows for declarative chain construction in LangChain, which simplifies building complex Large Language Model (LLM) applications. 

LCEL is useful because it supports streaming, async operations, and parallel execution, making it easier to build efficient and scalable LLM applications.
